# Graph DataBase <br>
그래프 DB는 NoSql 의 한 종류로 다양한 OpenSource 가 존재하지만, 그중 가장 유명한 neo4j 를 가지고 개발중인 F/W 에 적용하고자 기본적인 기능들은 Jupyter 로 사전에 테스트 하고자 한다. 개발 환경은 Python, Django 기반으로 "Neo4j Python Driver"라는 Python 라이브러리를 사용하여 Server 형태로 구축된 Graph Db 에 접속하여 일반적인 Node 생성, Relation 생성, Match 를 사용한 조회 작업등을 테스트할 예정이다. <br>
Neo4j 의 설치는 https://neo4j.com/download/community-edition/ 를 참조하면 간단하게 설치가 가능하며, 별다른 Setting 을 하지 않을 경우 기본 포트는 (api :7687, web:7474)가 된다. 아래의 예제들은 Neo4j가 설치되어 있다는 가정하에 수행된다.<br> 
<br><b>[그림1. graph data base]</b><br>
<img src="../images/graphdb.png"><br>


In [1]:
from neo4j.v1 import GraphDatabase, basic_auth
driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth("neo4j", "posco123!"))
session = driver.session()

# (1) Record 생성 
아래와 같이 Simple 하게 Lable 이 Person 이고 Property 가 name 인 Node를 만들어 보자. <br>

In [15]:
result = session.run("CREATE (you:Person {name:'You'})"
                     "RETURN you")
for record in result : 
    print("you : {0}".format(record["you"]))


you : <Node id=7 labels={'Person'} properties={'name': 'You'}>


# (2) Relation 생성 - 직업
Record 를 만들고 Relation 을 만들면 http://localhost:7474 에서 아래와 같은 그래프를 확인 할 수 있다.<br> 실행을 두번하여서 Person:you Node와 DataBase:Neo4j Node 사이에 관계가 Like 가 정의된 것을 확인 할 수 있다. <br> 
<img src="../images/graph_1.png"><br>

In [16]:
result = session.run("MATCH  (you:Person {name:'You'})"
                     "CREATE (you)-[like:LIKE]->(neo:Database {name:'Neo4j' })"
                     "RETURN you,like,neo")

for record in result : 
    print("{0}/{1}/{2}".format(record["you"], record["like"], record["neo"]))

<Node id=6 labels={'Person'} properties={'name': 'You'}>/<Relationship id=0 start=6 end=8 type='LIKE' properties={}>/<Node id=8 labels={'Database'} properties={'name': 'Neo4j'}>
<Node id=7 labels={'Person'} properties={'name': 'You'}>/<Relationship id=1 start=7 end=9 type='LIKE' properties={}>/<Node id=9 labels={'Database'} properties={'name': 'Neo4j'}>


# (3) Relation 생성 - 친구
아래와 같이 (you)-[:FRIEND]->(:Person {name:name} 관계를 추가적으로 생성하여 보자. <br>
아래와 같이 친구들이 추가된 것을 볼 수 있다. <br>
<img src="../images/graph_2.png"><br>

In [7]:
session.run("MATCH (you:Person {name:'You'})"
            "FOREACH (name in ['Johan','Rajesh','Anna','Julia','Andrew'] |"
            "  CREATE (you)-[:FRIEND]->(:Person {name:name}))")
result = session.run("MATCH (you {name:'You'})-[:FRIEND]->(yourFriends)"
                     "RETURN you, yourFriends")
for record in result :
    print("you : {0} , friends : {1}".format(record['you']['name'],record['yourFriends']['name'] ))

you : You , friends : Julia
you : You , friends : Anna
you : You , friends : Rajesh
you : You , friends : Johan
you : You , friends : Andrew
you : You , friends : Johan
you : You , friends : Rajesh
you : You , friends : Andrew
you : You , friends : Julia
you : You , friends : Anna
you : You , friends : Andrew
you : You , friends : Johan
you : You , friends : Rajesh
you : You , friends : Anna
you : You , friends : Julia
you : You , friends : Julia
you : You , friends : Anna
you : You , friends : Andrew
you : You , friends : Rajesh
you : You , friends : Johan
you : You , friends : Andrew
you : You , friends : Rajesh
you : You , friends : Johan
you : You , friends : Julia
you : You , friends : Anna
you : You , friends : Andrew
you : You , friends : Anna
you : You , friends : Julia
you : You , friends : Johan
you : You , friends : Rajesh
you : You , friends : Julia
you : You , friends : Anna
you : You , friends : Rajesh
you : You , friends : Johan
you : You , friends : Andrew
you : You , f

# (4) 친구의 친구 검색
위에서 만든 친구중에 Anna 라는 친구의 친구 Amanda 를 추가하고 Amanda 의 직업은 Neo4j 로 설정한다. <br>
그리고 당신의 친구중에 전문분야가 Neo4j 인 친구를 찾는 Path를 탐색한다. 

In [10]:
session.run("MATCH (neo:Database {name:'Neo4j'})"
            "MATCH (anna:Person {name:'Anna'})"
            "CREATE (anna)-[:FRIEND]->(:Person:Expert {name:'Amanda'})-[:WORKED_WITH]->(neo)")

result = session.run("MATCH (you {name:'You'})"
                     "MATCH (expert)-[:WORKED_WITH]->(db:Database {name:'Neo4j'})"
                     "MATCH path = shortestPath( (you)-[:FRIEND*..5]-(expert) )"
                     "RETURN db,expert,path")

for record in result :
    print("expert:{0}".format(record['expert'] ))
    print("path:{0}".format(record['path'] ))

expert:<Node id=162 labels={'Person', 'Expert'} properties={'name': 'Amanda'}>
path:<Path start=6 end=162 size=2>
expert:<Node id=164 labels={'Person', 'Expert'} properties={'name': 'Amanda'}>
path:<Path start=6 end=164 size=2>
expert:<Node id=166 labels={'Person', 'Expert'} properties={'name': 'Amanda'}>
path:<Path start=6 end=166 size=2>
expert:<Node id=168 labels={'Person', 'Expert'} properties={'name': 'Amanda'}>
path:<Path start=6 end=168 size=2>
expert:<Node id=154 labels={'Person', 'Expert'} properties={'name': 'Amanda'}>
path:<Path start=6 end=154 size=2>
expert:<Node id=156 labels={'Person', 'Expert'} properties={'name': 'Amanda'}>
path:<Path start=6 end=156 size=2>
expert:<Node id=158 labels={'Person', 'Expert'} properties={'name': 'Amanda'}>
path:<Path start=6 end=158 size=2>
expert:<Node id=160 labels={'Person', 'Expert'} properties={'name': 'Amanda'}>
path:<Path start=6 end=160 size=2>
expert:<Node id=132 labels={'Person', 'Expert'} properties={'name': 'Amanda'}>
path:<Pat

In [11]:
session.close()
print("close")